In [ ]:
from time import time
from pprint import pprint

import pandas as pd

from riotapi import *
from helper_functions import *

## Data

We get 100 players from diamond 1.

In [ ]:
summoner_name = "Unlucky Flip"

GAMES = 2
GAMES_PER_SUMMONER = 2

We then loop through all the players and gather their data about recent solo ranked games and save them as csv.

In [ ]:
# champion, role, lane, win, kills, deaths, assists, total_damage_dealt_to_champions, vision_score, gold_earned, total_minions_killed, game_time
match_list = []
teammates_set = set()
match_list_teammates = []


summoner_data = get_summoner_by_username(summoner_name)
account_id = summoner_data['accountId']
summoner_id = summoner_data['id']
summoner_win_rate = get_summoner_winrate(summoner_id)

match_references = get_match_references_by_account_id(account_id, 0, GAMES)

counter = 0
for match_reference in match_references:
    champion, role, lane, game_id = parse_match_reference(match_reference)
    game = get_match_by_id_safe(game_id)

    if game is not None:
        participant_data, win = parse_participant_data(game, champion, role, lane, summoner_win_rate)
        if counter < GAMES_PER_SUMMONER:
            match_list.append(participant_data)

        teammates = parse_teammates(game, champion)
        teammates_set.update(teammates)
        teammates.append(win)
        match_list_teammates.append(teammates)

save_match_to_csv(match_list, f'./data/{summoner_name}/match_lists/{summoner_name}.csv')
save_match_teammates_to_csv(match_list_teammates, f'./data/{summoner_name}/match_list_teammates.csv')
save_teammates_to_csv(teammates_set, f'./data/{summoner_name}/teammates.csv')

for teammate in teammates_set:
    match_list = []

    match_references = get_match_references_by_account_id(teammate, 0, GAMES_PER_SUMMONER)
    for match_reference in match_references:
        champion, role, lane, game_id = parse_match_reference(match_reference)
        game = get_match_by_id_safe(game_id)

        # TODO add SUMMONER_WINRATE


        if game is not None:
            participant_data, _ = parse_participant_data(game, champion, role, lane)
            match_list.append(participant_data)

    save_match_to_csv(match_list, f'./data/{summoner_name}/match_lists/{teammate}.csv')


print("Done!")